<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Introduction</a></span><ul class="toc-item"><li><span><a href="#Parametrized-SVD" data-toc-modified-id="Parametrized-SVD-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Parametrized SVD</a></span></li><li><span><a href="#Computing-the-SVD-derivatives" data-toc-modified-id="Computing-the-SVD-derivatives-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Computing the SVD derivatives</a></span></li></ul></li><li><span><a href="#Example:-Control-system" data-toc-modified-id="Example:-Control-system-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Example: Control system</a></span></li></ul></div>

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:85% !important; }</style>"))

import numpy
numpy.set_printoptions(edgeitems=30, linewidth=100000)

from matplotlib import pyplot as plt

%matplotlib notebook

# Introduction
The [singular value decompositon](https://en.wikipedia.org/wiki/Singular_value_decomposition) has many practical uses in science and engineering. It sates that any real $m\times n$ matrix $A$ can be written as $A = U\Sigma V^T$ where:
* $U$ and $V$ are [orthogonal matrices](https://en.wikipedia.org/wiki/Orthogonal_matrix) with shapes $m\times m$ and $n\times n$ respectively.
* $\Sigma$ is a positive diagonal matrix with shape $d\times d$ where $d=\min(m,n)$. The diagonal elements $\Sigma_{ii}\equiv\sigma_i\geq0$ are sorted in decending order so that $\sigma_1 \geq \ldots \geq \sigma_d$.

## Parametrized SVD

By taking a matrix $A(k)$ that depends on a parameter $k$ we can define functions $U(k)$, $\Sigma(k)$, $V(k)$ through the singular value decomposition $A(k) = U(k)\Sigma(k)V^T(k)$. It turns out that if $A(k)$ is continuous the components of the singular value decomposition are also continuous and differentiable with respect to $k$ in many cases*.

\*We have to assume there is a unique choice of $U(k), \Sigma(k)$ and $V(k)$ for each given $k$ which isn't the case. But practically the full trajectory should be well defined after fixing a signle point $U(k_0)$ as long as there are no repeated singular values (points where $\sigma_i = \sigma_j$ for $i\neq j$). See e.g. this [stack exchange thread](https://math.stackexchange.com/questions/644327/how-unique-are-u-and-v-in-the-singular-value-decomposition) for some interesting discussions on this topic. 

## Computing the SVD derivatives
It turns out that there's a relatively simple way to compute the derivatives of the SVD matrices with respect to $k$. The derivation can be found for example [here](https://projects.ics.forth.gr/_publications/2000_eccv_SVD_jacobian.pdf) and [here](https://j-towns.github.io/papers/svd-derivative.pdf) but I'll repeat it here with a notation that matches the python implementation in this repo. We start by implicitly differentiating both sides of the SVD relationship, letting $'$ denote derivative:
$$A' = U' \Sigma V^T + U \Sigma' V^T + U \Sigma V'^T$$
Then multiply by $U^T$ from the left and $V$ from the right and letting $R = U^TA' V$, $S^U = U^TU'$, $S^V = V'^TV$:
$$R = S^U\Sigma + \Sigma' + \Sigma S^V.$$

This means we have:
* $\sigma'_i = R_{ii}$ 
* $R_{ij} = S^U_{ij}\sigma_j + \sigma_i S^V_{ij}$ for $i\neq j$
* $R_{ji} = -\sigma_i S^U_{ij} - S^V_{ij} \sigma_j$ for $i\neq j$

The first equation gives us the derivative of $\sigma_i$! Relying on the assumption that all singular values have unique values we can solve the remaining two equations for $S^U_{ij}$ and $S^V_{ij}$ to get:
* $S^{U}_{ij} = (\sigma_jR_{ij} + \sigma_iR_{ji}) / (\sigma_j^2 - \sigma_i^2)$
* $S^{V}_{ij} = -(\sigma_iR_{ij} + \sigma_jR_{ji}) / (\sigma_j^2 - \sigma_i^2)$

Which can be written on matrix form as:
* $S^U = D\circ(R\Sigma + \Sigma R^T)$
* $S^V = -D\circ(\Sigma R + R^T\Sigma)$

where $D_{ij} = 1 / (\sigma_j^2 - \sigma_i^2)$ for $i\neq j$, $D_{ii} = 0$ and $\circ$ denotes elementwise multiplication.

We can then finally compute the derivatives of $U$ and $V$ using $U' = US^U$ and $V'^T = S^VV^T$.

# Example: Control system

As an illustrative example we consider a feedback system with input $x\in\mathrm{R}^n$ and output $y\in\mathrm{R}^m$ given by:
$$y = H(x + kFy)$$
where $k$ is a tunable scalar parameter and $H$, $F$ are fixed system matrices defined by some physical process.  Typically, $H$ and $F$ will depend on the complex frequency $s=\sigma + j\omega$ of the input but let's ignore that for now. 

Solving the above relationship for $y$ we get:
$$y = (1 - kHF)^{-1}Hx \equiv A(k)y$$
where $A(k)$ is the effective system matrix after taking the feedback control into account. A typical problem in control is to tune $k$ such that $A(k)$ has some desirable property like stability or response time for certain inputs. Such properties are often defined in terms of the singular value decomposition of $A(k)$ so being able to compute their derivatives with respect to could be interesting for e.g. sensitivity analysis.

To use the formulas above we need to first get a formula for $A'(k)$. We differentiate both sides of the relationship $(1-kHF)A(k) = H$ to get
$$(1-kHF)A'(k) - HFA(k) = 0$$
and thus $$A'(k) = (1-kHF)^{-1}HFA(k) = A(k)FA(k).$$

Below we verify that the SVD deritative corresponds to the numeric derivative for this example.

In [2]:
m, n = 10, 8
d = min(m, n)
H = numpy.random.normal(size=(m, n))
F = numpy.random.normal(size=(n, m))

def A(k):
    return numpy.linalg.inv(numpy.eye(m, m) - k * H.dot(F)).dot(H)

def A_dot(k):
    A_k = A(k)
    return A_k.dot(F).dot(A_k)

In [3]:
def svd_derivative_numeric(k, h):
    return [(numpy.linalg.svd(A(k+h), full_matrices=False)[i] - numpy.linalg.svd(A(k), full_matrices=False)[i]) / h for i in range(3)]

In [4]:
from svd_derivative import svd_derivative
for k in numpy.linspace(-10, 10, 10):
    A_k = A(k)
    A_dot_k = A_dot(k)

    u_dot, sigma_dot, vt_dot = svd_derivative(A_k, A_dot_k)
    u_dot_numeric, sigma_dot_numeric, vt_dot_numeric = svd_derivative_numeric(k, 1e-8)
    
    print(numpy.linalg.norm(u_dot - u_dot_numeric), numpy.linalg.norm(sigma_dot - sigma_dot_numeric), numpy.linalg.norm(vt_dot - vt_dot_numeric))

2.0318455814716085e-05 3.725357004821272e-07 2.1074138823252144e-05
2.064853046884153e-05 1.997718833669238e-06 1.9768722136139448e-05
1.1780055593274766e-05 6.833614173386992e-07 1.2000961501361443e-05
8.0072067968754e-06 2.1121936430638394e-06 7.763504037922919e-06
5.552156085229667e-06 4.410032183495064e-06 5.516681515540583e-06
3.0104531467658574e-06 4.0713415807812875e-07 2.5582883328278012e-06
1.5354702049301542e-05 1.532340321196321e-07 1.6794210608193617e-05
9.217260539825212e-06 8.540183625226665e-07 8.319937362753792e-06
2.35595003134097e-05 1.741036849996652e-06 2.155919506319858e-05
2.378968356367283e-05 8.485300040254726e-07 2.2870229212336187e-05
